___
<img style="float: right; margin: 15px 15px 15px 15px;" src="https://img.freepik.com/free-vector/depression-concept-illustration_114360-3747.jpg?t=st=1657678284~exp=1657678884~hmac=b8b1d71ca0a8eb2e4ff5bf31d6a98624112f1a2254b0f39e92254ed12d7875b2" width="240px" height="180px" />

# <font color= #bbc28d> **Clasificación de Depresión - Modelado** </font>
#### <font color= #2E9AFE> `Proyecto de Ciencia de Datos`</font>
- <Strong> Sofía Maldonado, Diana Valdivia, Samantha Sánchez & Vivienne Toledo </Strong>
- <Strong> Fecha </Strong>: 11/11/2025.

___

<p style="text-align:right;"> Image retrieved from: https://img.freepik.com/free-vector/depression-concept-illustration_114360-3747.jpg?t=st=1657678284~exp=1657678884~hmac=b8b1d71ca0a8eb2e4ff5bf31d6a98624112f1a2254b0f39e92254ed12d7875b2/p>

In [ ]:
# General Libraries
import os
import pandas as pd
# Databricks Env
import pathlib
import pickle
from dotenv import load_dotenv
# Feature Engineering
from sklearn.feature_extraction import DictVectorizer
# Optimization
import math
import optuna
from optuna.samplers import TPESampler
# MLFlow
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow import MlflowClient
# Modeling
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# Evaluation Metrics
from sklearn.metrics import root_mean_squared_error